In [3]:
import json
from time import time
import pandas as pd
from pathlib import Path
from dpmm.pipelines import AIMPipeline

/tmp/ipykernel_25606/1534794196.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Load data

In [4]:
wine_dir = Path().parent / "wine"

df = pd.read_pickle(wine_dir / "wine.pkl.gz")
with (wine_dir / "wine_bounds.json").open("r") as f:
    domain = json.load(f)



# Build AIM pipeline

In [5]:

pipeline = AIMPipeline(
    epsilon=1,  # Privacy budget for generate model
    proc_epsilon=0.1,  # Privacy budget for data processing 
    binner_type="priv-tree",  # Discretisation algorithm
    gen_kwargs={
        "n_iters": 1000,
        "degree": 2,
        "num_marginals": None,
        "max_cells": 10000,
    },
    delta=1e-5,  # Delta Setting in (eps, delta) differential privacu
    compress=True,  # Domain Compression paramter
    max_model_size=80,  # Maximum Model Size in Megabytes
    n_bins="auto"  # Number of bins
)

# Fit step

In [6]:
print("Fitting AIM pipeline...", end='\r')
start_time = time()
# The number of rows to generate
pipeline.fit(df, domain)
fit_time = time()
print(f"Fitting AIM pipeline - Took {fit_time - start_time:.2f} seconds")


Fitting AIM pipeline - Took 137.70 seconds


# Generate step

In [7]:
print("Generating synthetic data...", end="\r")
# Generate synthetic data
synth_df = pipeline.generate(df.shape[0])
gen_time = time()

print(f"Generating synthetic data - Took {gen_time - fit_time:.2f} seconds")
synth_df.head()

Generating synthetic data - Took 0.07 seconds


,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,9.794007,0.214221,0.263052,2.178182,0.030924,20.596338,93.423141,0.992662,2.991172,0.520225,11.106112,1
1,white,8.794490,0.319712,0.588624,2.223932,0.033643,9.581360,29.427055,0.993420,3.350735,1.215672,12.467395,1
2,white,6.166472,0.304872,0.351288,2.897107,0.014269,44.814246,100.761516,0.987278,3.413432,0.349481,12.156588,1
3,red,6.352416,0.426643,0.284893,2.092849,0.111270,7.964595,106.737013,0.999595,3.279719,1.030645,11.126675,1
4,white,8.100742,0.145250,0.535973,0.919340,0.030221,17.163646,159.919670,0.992147,3.259013,0.559875,13.033422,1
